In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense

from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys
import cv2
import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import pandas as pd

from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [2]:
img_width=128
img_height=128
is_learning=True

In [3]:
def get_model(nb_classes,img_width=128, img_height=128):
    input_tensor = Input(shape=(img_width, img_height, 3))
    # include_top=False 出力層なし
    model = ResNet50(include_top=False,classes=nb_classes, weights='imagenet',input_tensor=input_tensor)
    #  model = VGG16(include_top=False,classes=nb_classes, weights='imagenet',input_tensor=input_tensor)
    x=model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes, activation='softmax')(x)
    # VGG16とFC層を結合してモデルを作成（完成図が上の図）
    model= Model(input=model.input, output=x)

    # VGG16の図の青色の部分は重みを固定（frozen）
    for layer in model.layers[:15]:
        layer.trainable = False

    # 多クラス分類を指定
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])
    
    return model

In [4]:
resnet_model = get_model(2)

C:\Users\mopper\AppData\Local\conda\conda\envs\keras_work\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\mopper\AppData\Local\conda\conda\envs\keras_work\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [5]:
type(resnet_model)

keras.engine.training.Model

In [6]:
resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
# トレーンング用、バリデーション用データを生成するジェネレータ作成
train_datagen = ImageDataGenerator(
  rescale=1.0 / 255,
  #すでに画像の水増し済みの方は、下記２行は必要ありません。
  #zoom_range=0.2,
  #horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_datagen

In [8]:
# 訓練データ保存先
# /保存先フォルダ/ {0 or 1 } / *.png
classes=["0","1"]
classes_name =["輿水幸子","森久保乃々"]
train_data_dir="Y:/MLearningData/SatiMori/"
#トレーニングデータ用の画像数
#バッチサイズ
batch_size = 2


In [9]:
# 訓練データ
train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)

Found 3071 images belonging to 2 classes.


In [10]:
# テストデータ
validation_data_dir=  "Y:/MLearningData/SatiMori/test"
validation_generator = validation_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)

Found 54 images belonging to 2 classes.


In [11]:
# テストデータ（評価用
validation_data_dir=  "Y:/MLearningData/SatiMori/test"
validation_generator2 = validation_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=1,
  shuffle=False)

Found 54 images belonging to 2 classes.


In [12]:
#エポック数
nb_epoch = 2

In [13]:
# 訓練する場合
if is_learning:
    # Fine-tuning
    history = resnet_model.fit_generator(
        train_generator,
        # samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        # nb_val_samples=nb_validation_samples
    )
    vgg_model.save('res_model') 
    vgg_model.save_weights("res_model_weights")
else:
    vgg_model.load_weights("res_model_weights")


C:\Users\mopper\AppData\Local\conda\conda\envs\keras_work\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\mopper\AppData\Local\conda\conda\envs\keras_work\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=2)`
  


Epoch 1/2


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1/convolution (defined at C:\Users\mopper\AppData\Local\conda\conda\envs\keras_work\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_39475]

Function call stack:
keras_scratch_graph


In [ ]:
def write_cv(img,output_path,dummy_path="./tmp.jpg"):
    if cv2.imwrite(dummy_path, img)==False:
        return False
    if os.path.exists(output_path):
        os.remove(output_path)
    shutil.move(dummy_path,output_path)
    return True
    

In [ ]:
K.set_learning_phase(0)

In [ ]:
folder_path = "Y:/MLearningData/SatiMori/test/"
name_list=["輿水幸子","森久保乃々"]
prob_all =pd.DataFrame()
for i in range(2):
    glob_path = folder_path+str(i)+"/*.*"
    output_folder="./test/"+str(i)+"/"

    file_list = glob.glob(glob_path)
    
    predict0=[]
    predict1=[]
    predict2=[]
    predict2_name=[]
    ok_list=[]
    base_name_list =[]
    for file_path in file_list:
        print(file_path)
        base_name = os.path.basename(file_path)
        preprocessed_input_base = load_image(file_path)        
        p_image = preprocessed_input_base/255.0
        predict = vgg_model.predict(p_image,batch_size=1)
        print(predict)
        print(predict.shape)
        predict0.append(predict[0][0])
        predict1.append(predict[0][1])
        base_name_list.append(base_name)
        predict2.append(np.argmax(predict[0]))
        predict2_name.append(name_list[predict2[-1]])
        ok_list.append(np.argmax(predict[0])==i)
    prob = pd.DataFrame({name_list[0]:predict0,name_list[1]:predict1,"file_name":base_name_list})
    prob["答"]=int(i)
    prob["予測値"]=predict2
    prob["答_名"]=name_list[i]
    prob["予測名"]=predict2_name
    prob["正しく予測"]=ok_list
    prob_all = prob_all.append(prob)